In [24]:
import requests, json
import pandas as pd

In [42]:
def list_of_dicts_to_nested_dict(data):
    nested_dict = {}
    for item in data:
        current_dict = nested_dict
        for key, value in item.items():
            if key == 'agregados':
                if key not in current_dict:
                    current_dict[key] = []
                current_dict[key].extend(value)
            else:
                if key not in current_dict:
                    current_dict[key] = {}  # Create an empty nested dictionary if key doesn't exist
                current_dict = current_dict[key]
        if 'agregados' in current_dict:
            current_dict['agregados'] = list_of_dicts_to_nested_dict(current_dict['agregados'])
    return nested_dict

# Dados demográficos - IBGE

## População de Embu-Guaçu (2010-2022)

In [26]:
url_base = 'https://servicodados.ibge.gov.br/api/v3/agregados'

'https://servicodados.ibge.gov.br/api/v3/agregados/9605/periodos/2010%7C2022/variaveis/93?localidades=N6[3515103]&classificacao=86[all]'

'https://servicodados.ibge.gov.br/api/docs/agregados?versao=3'

'https://servicodados.ibge.gov.br/api/docs/agregados?versao=3'

## Listar dados disponíveis para município

In [27]:
# identificador nivel municipal
nivel = 'N6'

response = requests.get(f'{url_base}?nivel={nivel}')

data_json = response.json()

for conjunto_dataset in data_json:
    print(conjunto_dataset['nome'], conjunto_dataset['id'])

Áreas Urbanizadas D5
Cadastro Central de Empresas CL
Censo Agropecuário CA
Censo Demográfico CD
Contagem da População  CM
Estatísticas dos Cadastros de Microempreendedores Individuais D9
Estimativas de População XF
Fundações Privadas e Associações Sem Fins Lucrativos FP
Indicadores de Desenvolvimento Sustentável IU
Índice de Preços ao Consumidor em Real IL
Índice de Reajuste do Salário Mínimo IJ
Índice Nacional de Preços ao Consumidor PC
Índice Nacional de Preços ao Consumidor Amplo IA
Índice Nacional de Preços ao Consumidor Amplo 15 IQ
Índice Nacional de Preços ao Consumidor Amplo Especial IG
Índice Nacional de Preços ao Consumidor Especial PG
Objetivos de Desenvolvimento Sustentável C2
Pesquisa da Pecuária Municipal PP
Pesquisa de Estoques ES
Pesquisa de Informações Básicas Municipais PM
Pesquisa de Orçamentos Familiares OF
Pesquisa de Serviços de Hospedagem SH
Pesquisa Estatísticas do Registro Civil RC
Pesquisa Nacional de Saneamento Básico SB
Pesquisa Nacional de Saúde XN
Pesquisa 

## Filtro de dasets relevantes

In [28]:
datasets_selecionados = ['CD','CM','XF','PM','OF','SB','XN','AA']

In [45]:
conj_datasets_selecionados = [dataset for dataset in data_json if dataset['id'] in datasets_selecionados]

In [55]:
conj_datasets_selecionados[0]

{'id': 'CD',
 'nome': 'Censo Demográfico',
 'agregados': [{'id': '102',
   'nome': 'Mulheres de 10 anos ou mais de idade, que viviam em união conjugal, total e que tiveram filhos, e os filhos tidos pelas mulheres de 10 anos ou mais de idade, que viviam em união conjugal, os nascidos vivos, os nascidos vivos no período de referência de 12 meses, nascidos mortos e os que estavam vivos na data de referência, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres'},
  {'id': '103',
   'nome': 'Mulheres de 10 anos ou mais de idade, que viviam em união conjugal e tiveram filhos nascidos vivos, por número de filhos tidos nascidos vivos, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres'},
  {'id': '105',
   'nome': 'Filhos tidos nascidos vivos das mulheres de 10 anos ou mais de idade, que viviam em união conjugal, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulhere

## Listar anos disponíveis para cada conjunto de datasets

In [69]:
dict_anos_datasets = {}

for dataset in datasets_selecionados:
    list_anos = []

    response = requests.get(f'{url_base}?acervo=/S/{dataset}/P/Q')
    data_json = response.json()

    for ano in data_json:
        list_anos.append(ano['id'])

    dict_anos_datasets[dataset] = list_anos

In [70]:
dict_anos_datasets

{'CD': ['1872',
  '1890',
  '1900',
  '1920',
  '1940',
  '1950',
  '1960',
  '1970',
  '1980',
  '1991',
  '2000',
  '2010',
  '2022'],
 'CM': ['1996', '2007'],
 'XF': ['2001',
  '2002',
  '2003',
  '2004',
  '2005',
  '2006',
  '2008',
  '2009',
  '2011',
  '2012',
  '2013',
  '2014',
  '2015',
  '2016',
  '2017',
  '2018',
  '2019',
  '2020',
  '2021'],
 'PM': ['2005', '2020', '2021'],
 'OF': ['1987', '1995', '2002', '2008', '2018'],
 'SB': ['2000', '2008', '2017'],
 'XN': ['2013', '2019'],
 'AA': ['2009', '2012', '2015']}

In [72]:
# Neste caso, vamos trabalhar apenas com datasets que tiveram atualização recente
anos_datasets_selecionados = {'CD':['2010','2022'],'XF':'2021','PM':'2021','XN':'2019'}

## Listar tabelas disponíveis para os anos e datasets selecionados

In [84]:
for dataset in anos_datasets_selecionados.items():
    if isinstance(dataset[1], list):
        print(f'{url_base}?acervo=/S/{dataset[0]}/P/{dataset[1][0]}/P/{dataset[1][1]}/T/Q?_1700211710044')
        response = requests.get(f'{url_base}?acervo=/S/{dataset[0]}/P/1_{dataset[1][0]}/P/1_{dataset[1][1]}/T/Q?_1700211710044')
        data_json = response.json()
        print(data_json)
    else:
        periodo = f'1_{dataset[1]}'
        print(periodo)

https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/CD/P/2010/P/2022/T/Q?_1700211710044
[{'id': '1209', 'literal': 'População, por grupos de idade'}, {'id': '9605', 'literal': 'População residente, por cor ou raça, nos Censos Demográficos'}, {'id': '9606', 'literal': 'População residente, por cor ou raça, segundo o sexo e a idade'}, {'id': '9608', 'literal': 'População indígena, por grupos de idade e sexo'}, {'id': '9756', 'literal': 'Índice de envelhecimento, idade mediana e razão de sexo, por cor ou raça'}, {'id': '9757', 'literal': 'Índice de envelhecimento, idade mediana e razão de sexo da população indígena, segundo quesito de declaração'}]
1_2021
1_2021
1_2019


In [82]:
dataset[1]

'2019'

In [80]:
f'{url_base}?acervo=/S/{dataset[0]}/P/{dataset[1][0]}/P/{dataset[1][1]}/T/Q?_1700211710044'

'https://servicodados.ibge.gov.br/api/v3/agregados?acervo=/S/XN/P/2/P/0/T/Q?_1700211710044'